# 01 20

In [1]:
import numpy as np
from random import shuffle


In [2]:
data_input = ['{0:020b}'.format(i) for i in range(2**16)]
shuffle(data_input)
data_input = [map(int,i) for i in data_input]
ti = []
to = []
for i in data_input:
    temp_o = np.zeros((21))
    temp_list = []
    for j in i:
            temp_list.append([j])
    temp_list = np.array(temp_list)
    temp_o[np.sum(temp_list)] = 1
    ti.append(temp_list)
    to.append(temp_o)
data_input = ti
data_output = to

print(len(data_input))
[print(data_output[i], '\n',data_input[i].shape) for i in range(0,3)];

65536
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 (20, 1)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 (20, 1)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 (20, 1)


In [3]:
NUM_EXAMPLES = 10000
dev_input = data_input[NUM_EXAMPLES:]
dev_output = data_output[NUM_EXAMPLES:]

train_input = data_input[:NUM_EXAMPLES]
train_output = data_output[:NUM_EXAMPLES]

In [4]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  pass
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    #with tf.name_scope('stddev'):
    #  stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    # tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [6]:
import tensorflow as tf
tf.reset_default_graph()

LOGDIR = '/tmp/rnn1'
if tf.gfile.Exists(LOGDIR):
    tf.gfile.DeleteRecursively(LOGDIR)
tf.gfile.MakeDirs(LOGDIR)

with tf.name_scope('input'):
    data = tf.placeholder(tf.float32, shape=(None, 20, 1), name='x-input')
    target = tf.placeholder(tf.float32, shape=(None, 21), name='y-input')

num_hidden = 24

with tf.name_scope('rnn'):
    cell = tf.nn.rnn_cell.LSTMCell(num_units=num_hidden, state_is_tuple=True)
    val, state = tf.nn.dynamic_rnn(cell=cell, inputs=data, dtype=tf.float32)
    val = tf.transpose(val, [1,0,2])
    with tf.name_scope('lstm-last'):
        last = tf.gather(val, int(val.get_shape()[0]) - 1)
        variable_summaries(val)
        variable_summaries(last)

with tf.name_scope('fc'):
    with tf.name_scope('w'):
        weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]), name='weight')
        variable_summaries(weight)
    
    with tf.name_scope('b'):
        bias = tf.Variable(tf.constant(.1, shape=[target.get_shape()[1]]), name='bias')
        variable_summaries(bias)
        
    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
    
with tf.name_scope('cross_entropy'):    
    cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.02)
    minimize = optimizer.minimize(cross_entropy)

with tf.name_scope('accuracy'):
    mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
    error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
    tf.summary.scalar('error', error)

sess = tf.Session()

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)


merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(LOGDIR + '/train', sess.graph)
test_writer  = tf.summary.FileWriter(LOGDIR + '/test', sess.graph)
init_op = tf.global_variables_initializer()
sess.run(init_op)

batch_size = 512
no_of_batches = int(len(train_input)/batch_size)
epoch = 10
for i in range(epoch):
    ptr = 0
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        summary, _, error_train = sess.run([merged, minimize, error],{data: inp, target: out})
#         sess.run(minimize,{data: inp, target: out})
#                 options=run_options,
#                 run_metadata=run_metadata)
    error_dev = sess.run(error,{data: dev_input, target: dev_output})
    # error_train = sess.run(error,{data: train_input, target: train_output})
    # train_writer.add_run_metadata(run_metadata, 'epoch%03d' % i)
    train_writer.add_summary(summary, i)
    print("Epoch %d : train error=%.6f   -  dev error=%.6f" % (i, error_train, error_dev))
# print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
sess.close()
train_writer.close()
test_writer.close()

/Users/larrysu/.pyenv/versions/p3lab/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 0 : train error=0.804688   -  dev error=0.798833
Epoch 1 : train error=0.636719   -  dev error=0.631608
Epoch 2 : train error=0.544922   -  dev error=0.631878
Epoch 3 : train error=0.216797   -  dev error=0.345866
Epoch 4 : train error=0.271484   -  dev error=0.244850
Epoch 5 : train error=0.187500   -  dev error=0.211286
Epoch 6 : train error=0.113281   -  dev error=0.116879
Epoch 7 : train error=0.054688   -  dev error=0.089041
Epoch 8 : train error=0.037109   -  dev error=0.218759
Epoch 9 : train error=0.027344   -  dev error=0.036643
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.03 0.65 0.26 0.05
  0.01 0.   0.   0.   0.   0.   0.  ]]
